# Setup
1. install python (3.8) and git
2. install virtualenv e.g. with `<path to python> -m to pip install virtualenv`
3. create a virtual environment that uses python 3.8 using virtualenv e.g. with `<path to python> -m virtualenv "<path to virtual environment>"`
4. activate the virtual environment e.g. on windows with `.\<pathto env>\Scripts\activate`
5. install vscode and python extension
6. git clone brainlit (https://github.com/neurodata/brainlit) and neuroglancer (https://github.com/google/neuroglancer)
7. install brainlit from source e.g. `pip install -e .` in brainlit folder 

In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr
import zarr
from cloudvolume import CloudVolume
import json

# Convert czi to zarr

In [ ]:
project_path = "/cis/project/sriram/" # "C:\\Users\\Sriram Sudarsanam\\Desktop\\NeuroglancerTrial\\"
czi_path = f"{project_path}Sriram/SS IUE 175 SNOVA RFP single channel AdipoClear Brain 1 full Image 3.czi" # path to czi image
out_dir = f"{project_path}ng_data/sriram-adipo-brain1-im3/" # path to directory where zarr should be made, should end in slash

In [ ]:
zarr_paths = czi_to_zarr(czi_path=czi_path, out_dir=out_dir)

# Convert zarr to ome-zarr

In [ ]:
out_path = f"{project_path}ng_data/sriram-adipo-brain1-im3/fg_ome.zarr" # path of ome zarr to be made

In [ ]:
zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=out_path)

# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/cis/project/sriram/ng_data/" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/sriram-adipo-brain1-im3/fg_ome.zarr

# Add trace layer

In [ ]:
traces_dir = f"{out_dir}traces/"
z = zarr.open(out_dir + "fg.zarr")
volume_size = z.shape
chunk_size = z.chunks
resolution = [1000000000, 1000000000, 1000000000]

outpath = (
    f"precomputed://file://{traces_dir}"
)

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16", 
    encoding="raw",
    resolution=resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=chunk_size,  # units are voxels
    volume_size=volume_size,  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(outpath, info=info, compress=False)
vol.commit_info()
vol.skeleton.meta.commit_info()

# remove vertex type attribute because it is a uint8 and incompatible with neuroglancer
info_path = f"{traces_dir}skeletons/info"
with open(info_path) as f:
    data = json.load(f)
    for i, attr in enumerate(data['vertex_attributes']):
        if attr['id'] == 'vertex_types':
            data['vertex_attributes'].pop(i)
            break

with open(info_path, "w") as f:
    json.dump(data, f)